In [4]:
class data:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
args = data()
args.filename = "paultan.csv"
args.stemmer = "lemma"
args.dictionary = "automotive"
args.model = "lda"

dir = os.getcwd()

In [5]:
# -*- coding: utf-8 -*-
import numpy as np
import lda
import lda.datasets
import csv
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from gensim import corpora, models

from stemming.porter2 import stem
from nltk.stem import *
import unicodecsv
import re
import os
import pyLDAvis.gensim
import gensim

_digits = re.compile('\d')
def contains_digits(d):
    return bool(_digits.search(d))

import enchant
d = enchant.Dict("en_US")
# Or using the /usr/share/dict/british-english word list
with open(args.dictionary + "-english") as word_file:
  english_words = set(word.strip().lower() for word in word_file)
  print(english_words)
  def is_english_word(word):
    return word.lower() in english_words

def process_tokens(tokens,stemmer):
  tokens = [i for i in tokens if not i in en_stop and not contains_digits(i) and is_english_word(i)]
  if stemmer == 'porter':
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(i) for i in tokens]
  elif stemmer == 'porter2':
    tokens = [stem(i) for i in tokens]
  elif stemmer == 'lemma':
    lemmatiser = WordNetLemmatizer()
    tokens = [lemmatiser.lemmatize(i) for i in tokens]
  return tokens


if args.filename == "sample":
  X = lda.datasets.load_reuters()
  dictionary = lda.datasets.load_reuters_vocab()
  titles = lda.datasets.load_reuters_titles()
else:
  dataset_filepath = os.path.join(dir, 'datasets/' + args.filename)
  f = open(dataset_filepath)
  reader = unicodecsv.reader(f, encoding='utf-8')
  # csv_length = sum(1 for row in reader)
  # f.seek(0) #reset reader position
  identifiers = reader.next()
  contents_idx = identifiers.index("contents")
  title_idx = identifiers.index("title")

  contents = [ row[contents_idx] for row in reader if row[contents_idx] ]

  f.seek(0)
  reader.next()
  titles = [ row[title_idx] for row in reader if row[contents_idx] ]



  texts = list()
  tokenizer = RegexpTokenizer(r'\w+')
  en_stop = get_stop_words('en')
  for idx,i in enumerate(contents):
#     if not idx % 10:
#       print "INFO: Tokenizing articles <{}> ".format(idx)
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    texts.append(process_tokens(tokens, args.stemmer))
    # print idx
    # add tokens to list

  print "[DEBUG] Length of Texts : {}".format(len(texts))
  dictionary = corpora.Dictionary(texts)
  corpus = [dictionary.doc2bow(text) for text in texts]


set(['changfeng motor', 'hi-test', 'magnetic', 'alternatives', 'dynasty', 'obstruction', 'bonneville salt flats', 'query', 'loyalty', 'adjustment', 'integrity', 'gonow', 'synthetic', 'saleswoman', 'disposal', 'technique', 'all-wheel drive', 'corrosive', 'jalopy', 'jianghuai', 'safety', 'infant seats', 'gravel', 'roadway', 'protective', 'lessons', 'advantage', 'dynamic', 'activation', 'aston martin', 'faw', 'mph', 'liability', 'balancing', 'drinking', 'lubrication', 'courageous', 'condition', 'tickets', 'retrial', 'transmission', 'race', 'jiangnan', 'recall', 'quick', 'inspection', 'accumulation', 'triumph', 'revolution', 'occurrence', 'corvette', 'four door', 'horn', 'tires', 'cost', 'design', 'enhancement', 'rigorous', 'runner-up', 'wings', 'hongqi', 'wreck', 'fine', 'specification', 'overload', 'sequential', 'optional', 'roewe', 'international', 'changhe', 'appeal', 'racing', 'ignition', 'impact', 'zhongyu', 'jiangling', 'component', 'chassis', 'lubricant', 'hwanghai', 'pedals', 'mac

In [7]:
pyLDAvis.enable_notebook()
model_filename = os.path.join(dir, 'models/' + args.filename.split('.')[0] + "_" + args.model + '.model')
try:
  ldamodel = models.LdaModel.load(model_filename)
except IOError:
  if(args.model == "lda"):
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=20)
  elif(args.model == "dtm"):
    my_timeslices = [500,500,500,500,500,346]
    ldamodel = gensim.models.wrappers.DtmModel('/Users/Hii/Projects/news_scraper/dtm-darwin64', corpus, my_timeslices, num_topics=20, id2word=dictionary,initialize_lda=True)
  else:
    raise ValueError('Unknown Model Type')
  ldamodel.save(model_filename)


# ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=20)
# pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)



/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:672: DeprecationWarning: PlainTextFormatter._type_printers_default is deprecated: use @default decorator instead.
  def _type_printers_default(self):


In [17]:
  for idx, topic in enumerate(ldamodel.show_topics(topics=10, topn=10, log=False, formatted=False)):
    print "Topic #" + str(idx) + " :",
    for word in topic:
      print word[1],
    print

Topic #0 : year safety increase price car impact point vehicle test cost
Topic #1 : volkswagen perodua year recall scandal brand vehicle automaker model new
Topic #2 : audi drive cockpit new navigation lot comfort map showroom information
Topic #3 : car porsche speed power engine drive weight wheel performance chassis
Topic #4 : design ferrari fiat bentley jeep car year feature new dashboard
Topic #5 : engine speed new transmission automatic torque manual cylinder model mitsubishi
Topic #6 : toyota vehicle new accessory year car part technical passenger fuel
Topic #7 : engine fuel new power efficiency performance oil consumption cylinder technology
Topic #8 : new car model sedan year design debut feature make engine
Topic #9 : control feature new speed display drive safety leather automatic model
Topic #10 : year safety increase price car impact point vehicle test cost
Topic #11 : volkswagen perodua year recall scandal brand vehicle automaker model new
Topic #12 : audi drive cockpit ne

In [8]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)



/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:672: DeprecationWarning: PlainTextFormatter._type_printers_default is deprecated: use @default decorator instead.
  def _type_printers_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      33.268599        1       1 -0.169800 -0.107816
7      16.355853        1       2 -0.042467 -0.055386
5      10.639171        1       3  0.160053 -0.008274
3       9.040918        1       4  0.203075 -0.107226
9       7.477591        1       5  0.093349 -0.064371
0       6.880461        1       6 -0.087671 -0.114443
1       4.385165        1       7 -0.249648  0.096370
8       4.346273        1       8 -0.004040  0.268706
2       4.303658        1       9 -0.040547 -0.060764
6       3.302311        1      10  0.137697  0.153204, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
355   Default  1489.000000           bmw  1489.000000  30.0000  30.0000
338   Default   891.000000        proton   891.000000  29.0000  29.0000
290   Default  1054.000000         honda  1054.000000  28.0000  28.0000
436   Default   637.000000        nissan   637.000000  27.0000  27.0000
500   Default   610.000000          ford   610.000000  26.0000  26.0000
373   Default   842.000000          audi   842.000000  25.0000  25.0000
27    Default  3599.000000           car  3599.000000  24.0000  24.0000
437   Default  1067.000000        toyota  1067.000000  23.0000  23.0000
463   Default   773.000000    volkswagen   773.000000  22.0000  22.0000
317   Default   631.000000         mazda   631.000000  21.0000  21.0000
282   Default   878.000000          fuel   878.000000  20.0000  20.0000
252   Default   972.000000         class   972.000000  19.0000  19.0000
295   Default  4924.000000           new  4924.000000  18.0000  18.0000
148   Default   998.000000         wheel   998.000000  17.0000  17.0000
206   Default   609.000000       service   609.000000  16.0000  16.0000
156   Default  1940.000000          year  1940.000000  15.0000  15.0000
26    Default   427.000000          race   427.000000  14.0000  14.0000
173   Default   437.000000       highway   437.000000  13.0000  13.0000
389   Default   838.000000        driver   838.000000  12.0000  12.0000
251   Default   621.000000         sedan   621.000000  11.0000  11.0000
37    Default  1173.000000        design  1173.000000  10.0000  10.0000
177   Default   380.000000       hyundai   380.000000   9.0000   9.0000
351   Default  2312.000000        engine  2312.000000   8.0000   8.0000
493   Default   278.000000        suzuki   278.000000   7.0000   7.0000
486   Default   649.000000         coupe   649.000000   6.0000   6.0000
168   Default   846.000000        safety   846.000000   5.0000   5.0000
395   Default   345.000000           kia   345.000000   4.0000   4.0000
124   Default  1508.000000       feature  1508.000000   3.0000   3.0000
346   Default   355.000000       porsche   355.000000   2.0000   2.0000
46    Default   311.000000        racing   311.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
181   Topic10     3.788387    regulatory    15.681590   1.9900  -6.5433
293   Topic10    14.809449         joint    62.316503   1.9736  -5.1799
449   Topic10    30.072260   competition   136.699278   1.8964  -4.4716
467   Topic10    21.315829  manufacturer   119.733575   1.6847  -4.8158
531   Topic10    55.909495         event   335.517736   1.6186  -3.8515
243   Topic10     0.706877      accuracy     4.714561   1.5130  -8.2221
462   Topic10     5.636068      schedule    38.898590   1.4788  -6.1460
359   Topic10    17.552112         value   123.359870   1.4606  -5.0100
311   Topic10     5.084776          pump    35.875111   1.4568  -6.2490
55    Topic10    26.980875     technical   195.342246   1.4309  -4.5801
13    Topic10     5.104289     advantage    39.342766   1.3683  -6.2451
384   Topic10    37.399676   development   322.175195   1.2571  -4.2536
554   Topic10    23.115356      original   220.486201   1.1552  -4.73